In [5]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [6]:

# Import Info
DF_YFIN_INFO1 = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_INFO_1.csv',sep='|') 
DF_YFIN_INFO2 = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_INFO_2.csv',sep='|') 
DF_YFIN_INFO3 = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_INFO_3.csv',sep='|') 
DF_YFIN_INFO4 = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_INFO_4.csv',sep='|') 

DF_YFIN_INFO = pd.concat([DF_YFIN_INFO1, DF_YFIN_INFO2, DF_YFIN_INFO3, DF_YFIN_INFO4], ignore_index=True)
# DF_YFIN_INFO = pd.concat([DF_YFIN_INFO1, DF_YFIN_INFO2, DF_YFIN_INFO3], ignore_index=True)


del DF_YFIN_INFO1, DF_YFIN_INFO2, DF_YFIN_INFO3, DF_YFIN_INFO4

DF_YFIN_INFO_VARS = list(DF_YFIN_INFO.columns)


In [37]:
# Keep key variables


INF_SUBSET = DF_YFIN_INFO[[
'symbol','sharesOutstanding',
'totalRevenue','revenueQuarterlyGrowth','revenueGrowth','grossMargins','grossProfits',
'currentRatio','totalAssets',
'dividendRate','dividendYield','fiveYearAvgDividendYield',
'earningsGrowth','earningsQuarterlyGrowth',
'marketCap',
'trailingPE',
'industry','sector',

# add variations for v2

# Price to book of under 1.2
'priceToBook',

# debt to asset less than 1.1
'totalDebt','totalAssets'



# Price / volume Movement	
# '52WeekChange','fiftyDayAverage','fiftyTwoWeekHigh','fiftyTwoWeekLow','averageDailyVolume10Day','averageVolume10days',
# Dividend	
# 'dividendRate','dividendYield','exDividendDate','payoutRatio','trailingAnnualDividendRate','trailingAnnualDividendYield',
# Timing	
# 'mostRecentQuarter','lastFiscalYearEnd','lastSplitDate','nextFiscalYearEnd',
# Fundamentals	
# 'open','marketCap','sharesOutstanding','floatShares','bookValue','regularMarketPrice','regularMarketVolume','heldPercentInsiders','heldPercentInstitutions',
# Performance	
# 'earningsQuarterlyGrowth','netIncomeToCommon','beta','enterpriseToEbitda','enterpriseToRevenue','enterpriseValue','priceToBook','priceToSalesTrailing12Months','profitMargins','trailingEps','trailingPE','fullTimeEmployees',
# Future dated	
# 'forwardEps','forwardPE',

]]

INF_SUBSET.rename(columns={"symbol": "TickName"}, inplace = True)





In [40]:
# Apply criteria
# start with defensive stocks

BG_CONS = INF_SUBSET

BG_CONS['FLAG_2_SALES'] = np.where(BG_CONS['totalRevenue'] > 1000000000 , 1, 0)
BG_CONS['FLAG_3_CURR_RATIO'] = np.where(BG_CONS['currentRatio'] > 1.5 , 1, 0)
BG_CONS['FLAG_4_DIV'] = np.where((BG_CONS['dividendYield'] ) > 0, 1, 0)
BG_CONS['FLAG_4_DIV_GTH'] = 0
# INF_SUBSET['FLAG_4_DIV_GTH'] = np.where((INF_SUBSET['fiveYearAvgDividendYield']) > 0, 1, 0)


BG_CONS['FLAG_5_ERN'] = np.where(BG_CONS['grossProfits'] > 0 , 1, 0)
BG_CONS['FLAG_6_ERN_GTH'] = np.where(BG_CONS['earningsGrowth'] > .02 , 1, 0)

# 7. Market cap < (asset - liability) * 1.5

BG_CONS['FLAG_7_MKT_CAP'] = np.where(BG_CONS['marketCap'] > .02 , 1, 0)

BG_CONS['FLAG_8_PE'] = np.where(BG_CONS['trailingPE'] < 15 , 1, 0)

BG_CONS['FLAG_BG_SCORE'] = BG_CONS['FLAG_2_SALES'] + BG_CONS['FLAG_3_CURR_RATIO'] + BG_CONS['FLAG_4_DIV']+ BG_CONS['FLAG_4_DIV_GTH']+ BG_CONS['FLAG_5_ERN']+ BG_CONS['FLAG_6_ERN_GTH']+ BG_CONS['FLAG_7_MKT_CAP']


BG_CONS_SUM = BG_CONS.groupby(['FLAG_BG_SCORE'])['TickName'].count().reset_index()

BG_CONS.to_excel('/Users/joezhou/Downloads/BG_CONSERVATIVE.xlsx', engine='xlsxwriter')  



In [39]:
# Apply criteria
# start with defensive stocks

BG_ENT = INF_SUBSET

BG_ENT['FLAG_2_SALES'] = np.where(BG_ENT['revenueGrowth'] > 0 , 1, 0)
BG_ENT['FLAG_3_CURR_RATIO'] = np.where(BG_ENT['currentRatio'] > 1 , 1, 0)
BG_ENT['FLAG_4_DIV'] = 0
INF_SUBSET['FLAG_4_DIV_GTH'] = np.where((INF_SUBSET['fiveYearAvgDividendYield']) > 0, 1, 0)


BG_ENT['FLAG_5_ERN'] = np.where(BG_ENT['grossProfits'] > 0 , 1, 0)
BG_ENT['FLAG_6_ERN_GTH'] = np.where(BG_ENT['earningsGrowth'] > .02 , 1, 0)

# 7. Market cap < (asset - liability) * 1.5

BG_ENT['FLAG_7_MKT_CAP'] = np.where(BG_ENT['marketCap'] > .02 , 1, 0)

BG_ENT['FLAG_8_PE'] = np.where(BG_ENT['trailingPE'] < 25 , 1, 0)

BG_ENT['FLAG_BG_SCORE'] = BG_ENT['FLAG_2_SALES'] + BG_ENT['FLAG_3_CURR_RATIO'] + BG_ENT['FLAG_4_DIV']+ BG_ENT['FLAG_4_DIV_GTH']+ BG_ENT['FLAG_5_ERN']+ BG_ENT['FLAG_6_ERN_GTH']+ BG_ENT['FLAG_7_MKT_CAP']


BG_ENT_SUM = BG_ENT.groupby(['FLAG_BG_SCORE'])['TickName'].count().reset_index()

BG_ENT.to_excel('/Users/joezhou/Downloads/BG_ENTERPRISING.xlsx', engine='xlsxwriter')  



In [41]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

# HIST_PRICE_DF = pd.read_excel (r'/Users/joezhou/Downloads/ALL_Prices.xlsx')
HIST_PRICE_DF_RAW = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_Prices.csv', sep='|')

#Create a sample for code testing purposes
# HIST_PRICE_DF = HIST_PRICE_DF_RAW[HIST_PRICE_DF_RAW['TickName'].isin(['BHP.AX','RIO.AX','TCL.AX','CBA.AX','MQG.AX','CSL.AX','NAB.AX','WBC.AX','SCG.AX','ANZ.AX','FMG.AX','WES.AX','TLS.AX','RMD.AX','WOW.AX','APA.AX','ATM.AX','GMG.AX','STO.AX','AMC.AX','MGR.AX','COL.AX','ALL.AX','NCM.AX','ZIP.AX'])]

HIST_PRICE_DF = HIST_PRICE_DF_RAW

#create a baselist for adding on the feature engineered variables
TICKER_LIST = HIST_PRICE_DF['TickName'].drop_duplicates().reset_index(drop=True)

In [42]:

import numpy as np

# find ends of the data window
HIST_PRICE_DF_FIRST = HIST_PRICE_DF.groupby("TickName").first()
HIST_PRICE_DF_FIRST.rename(columns={"Open": "First_Open_price"}, inplace = True)


HIST_PRICE_DF_LAST = HIST_PRICE_DF.groupby("TickName").last()
HIST_PRICE_DF_LAST.rename(columns={"Close": "Last_Close_price"}, inplace = True)
HIST_PRICE_DF = HIST_PRICE_DF.merge(HIST_PRICE_DF_LAST[['Last_Close_price']], on = 'TickName',how = 'left')


# create calculated columns
HIST_PRICE_DF['Freq'] = 1

HIST_PRICE_DF['DAY_UP'] = np.where(HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] > 0, 1, 0)
HIST_PRICE_DF['DAY_DWN'] = np.where(HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] < 0, 1, 0)
HIST_PRICE_DF['DAY_FLAT'] = np.where(HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] == 0, 1, 0)

HIST_PRICE_DF['DAY_CHANGE_RATE'] = HIST_PRICE_DF['Close'] / HIST_PRICE_DF['Open'] - 1
HIST_PRICE_DF['DAY_CHANGE_NUM'] = HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] 

HIST_PRICE_DF['DAYS_ABOVE_LCLSE'] = np.where(HIST_PRICE_DF['Last_Close_price'] - HIST_PRICE_DF['Close'] > 0, 1, 0)
HIST_PRICE_DF['DAYS_BELOW_LCLSE'] = np.where(HIST_PRICE_DF['Last_Close_price'] - HIST_PRICE_DF['Close'] < 0, 1, 0)

# aggregate to ticker level for calcs
HIST_PRICE_DF_SUM = HIST_PRICE_DF.groupby("TickName")["Freq", "DAY_UP","DAY_DWN", "DAY_FLAT", "DAY_CHANGE_NUM", "DAY_CHANGE_RATE", "DAYS_ABOVE_LCLSE", "DAYS_BELOW_LCLSE"].sum()



<ipython-input-42-a80d6e694314>:27: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  HIST_PRICE_DF_SUM = HIST_PRICE_DF.groupby("TickName")["Freq", "DAY_UP","DAY_DWN", "DAY_FLAT", "DAY_CHANGE_NUM", "DAY_CHANGE_RATE", "DAYS_ABOVE_LCLSE", "DAYS_BELOW_LCLSE"].sum()


In [43]:
# create ticker level file

DF_PRICE_INFO = HIST_PRICE_DF_SUM[["Freq", "DAY_UP","DAY_DWN", "DAY_FLAT", "DAYS_ABOVE_LCLSE", "DAYS_BELOW_LCLSE"]].merge(HIST_PRICE_DF_FIRST[['First_Open_price']], on = 'TickName',how = 'left')
DF_PRICE_INFO = DF_PRICE_INFO.merge(HIST_PRICE_DF_LAST[['Last_Close_price']], on = 'TickName',how = 'left')

# calculate at ticker level
DF_PRICE_INFO['PROB_HIGH'] = DF_PRICE_INFO['DAY_UP'] / DF_PRICE_INFO['Freq']
DF_PRICE_INFO['PROB_DWN'] = DF_PRICE_INFO['DAY_DWN'] / DF_PRICE_INFO['Freq']
DF_PRICE_INFO['PROB_FLAT'] = DF_PRICE_INFO['DAY_FLAT'] / DF_PRICE_INFO['Freq']

DF_PRICE_INFO['PROB_DAY_ABOVE_LCLSE'] = DF_PRICE_INFO['DAYS_ABOVE_LCLSE'] / DF_PRICE_INFO['Freq']
DF_PRICE_INFO['PROB_DAY_BELOW_LCLSE'] = DF_PRICE_INFO['DAYS_BELOW_LCLSE'] / DF_PRICE_INFO['Freq']

DF_PRICE_INFO['CHG_RTE_PRICE'] = DF_PRICE_INFO['Last_Close_price'] / DF_PRICE_INFO['First_Open_price'] -1

del HIST_PRICE_DF_SUM, HIST_PRICE_DF_FIRST, HIST_PRICE_DF_LAST

In [44]:
DF_PRICE_INFO.to_excel('/Users/joezhou/Downloads/Pricing_Inf_latest.xlsx', engine='xlsxwriter')  
